# 09.01 – Building Our LLM Handler
Let's bridge the gap between our rule-based chatbot and connecting to real LLMs by building our own handler step-by-step! 🔨

## ✅ Goals
- Understand what an LLM handler does
- Build a handler class from scratch
- Add support for multiple LLM providers
- Implement conversation history tracking

## 🤔 Why Do We Need an LLM Handler?

Our rule-based chatbot from the previous notebook was limited to pre-defined responses. To create a more dynamic and intelligent chatbot, we need to connect to Large Language Models (LLMs).

An LLM handler helps us:
1. Connect to different LLM providers (like OpenAI or local models)
2. Format prompts correctly
3. Track conversation history
4. Handle API calls and responses

## 🔍 Step 1: Setting Up the Environment

First, let's import the libraries we'll need:

In [ ]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()  # This will load environment variables from a .env file

# Test if our environment is set up correctly
print(f"OpenAI API Key set: {'Yes' if os.getenv('OPENAI_API_KEY') else 'No'}")

## 🏗️ Step 2: Creating the Basic Handler Class

Let's start with a simple version of our LLM handler:

In [ ]:
class BasicLLMHandler:
    def __init__(self, name="PyPal", role="helpful AI assistant"):
        self.name = name
        self.role = role
        self.system_prompt = f"Your name is {name}, and you are a {role}. Always respond accordingly to this role."
    
    def generate(self, prompt):
        # For now, just return a mock response
        return f"[{self.name}]: I am a {self.role} and you said: {prompt}"

# Test our basic handler
basic_handler = BasicLLMHandler(name="Buddy", role="Python tutor")
response = basic_handler.generate("Hello, can you help me learn Python?")
print(response)

## 🧠 Step 3: Adding Conversation History

Let's enhance our handler to keep track of the conversation history:

In [ ]:
class LLMHandlerWithHistory:
    def __init__(self, name="PyPal", role="helpful AI assistant"):
        self.name = name
        self.role = role
        self.system_prompt = f"Your name is {name}, and you are a {role}. Always respond accordingly to this role."
        self.history = []
    
    def add_to_history(self, role, content):
        """Add a message to the conversation history."""
        self.history.append({"role": role, "content": content})
    
    def show_history(self):
        """Display the conversation history."""
        for entry in self.history:
            print(f"{entry['role'].capitalize()}: {entry['content']}")
    
    def generate(self, prompt):
        """Generate a response to the given prompt."""
        # Add user message to history
        self.add_to_history("user", prompt)
        
        # For now, just return a mock response
        response = f"I am {self.name} the {self.role}. You said: {prompt}"
        
        # Add assistant response to history
        self.add_to_history("assistant", response)
        
        return response

# Test our handler with history
history_handler = LLMHandlerWithHistory(name="Alex", role="coding coach")
print(history_handler.generate("Hi there!"))
print(history_handler.generate("Can you help me with Python?"))
print("\nConversation History:")
history_handler.show_history()

## 🔌 Step 4: Connecting to OpenAI

Now let's add functionality to connect to OpenAI's API:

In [ ]:
class OpenAIHandler:
    def __init__(self, model="gpt-4o-mini", name="PyPal", role="helpful AI assistant"):
        self.api_key = os.getenv("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OPENAI_API_KEY environment variable is required")
            
        self.model = model
        self.name = name
        self.role = role
        self.system_prompt = f"Your name is {name}, and you are a {role}. Always respond accordingly to this role."
        self.history = []
    
    def add_to_history(self, role, content):
        self.history.append({"role": role, "content": content})
    
    def show_history(self):
        for entry in self.history:
            print(f"{entry['role'].capitalize()}: {entry['content']}")
    
    def generate(self, prompt):
        """Generate a response using OpenAI's API."""
        url = "https://api.openai.com/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        # Get recent history (last 3 messages)
        recent = self.history[-3:] if len(self.history) >= 3 else self.history[:]
        
        # Prepare messages
        messages = [
            {"role": "system", "content": self.system_prompt}
        ]
        
        # Add conversation history
        messages.extend(recent)
        
        # Add current user message
        messages.append({"role": "user", "content": prompt})
        
        # Prepare the request data
        data = {
            "model": self.model,
            "messages": messages
        }
        
        try:
            # Make API request
            response = requests.post(url, headers=headers, json=data)
            response.raise_for_status()  # Raise exception for HTTP errors
            
            # Parse the response
            result = response.json()
            reply = result["choices"][0]["message"]["content"]
            
            # Add to history
            self.add_to_history("user", prompt)
            self.add_to_history("assistant", reply)
            
            return reply
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            print(error_msg)
            return "Sorry, I encountered an error while generating a response."

# Note: We won't run this yet because it requires a valid API key
# openai_handler = OpenAIHandler()
# print(openai_handler.generate("Hello, how are you today?"))

## 🌐 Step 5: Supporting Local Models with Ollama

Let's add support for local models using Ollama:

In [ ]:
class OllamaHandler:
    def __init__(self, model="llama3", name="PyPal", role="helpful AI assistant"):
        self.model = model
        self.name = name
        self.role = role
        self.system_prompt = f"Your name is {name}, and you are a {role}. Always respond accordingly to this role."
        self.history = []
    
    def add_to_history(self, role, content):
        self.history.append({"role": role, "content": content})
    
    def show_history(self):
        for entry in self.history:
            print(f"{entry['role'].capitalize()}: {entry['content']}")
    
    def generate(self, prompt):
        """Generate a response using Ollama's API."""
        url = "http://localhost:11434/api/chat"
        
        # Get recent history (last 3 messages)
        recent = self.history[-3:] if len(self.history) >= 3 else self.history[:]
        
        # Prepare messages
        messages = [
            {"role": "system", "content": self.system_prompt}
        ]
        
        # Add conversation history
        messages.extend(recent)
        
        # Add current user message
        messages.append({"role": "user", "content": prompt})
        
        # Prepare the request data
        data = {
            "model": self.model,
            "messages": messages,
            "stream": False  # We don't want streaming for now
        }
        
        try:
            # Make API request
            response = requests.post(url, json=data)
            response.raise_for_status()  # Raise exception for HTTP errors
            
            # Parse the response
            result = response.json()
            reply = result.get("message", {}).get("content", "No reply")
            
            # Add to history
            self.add_to_history("user", prompt)
            self.add_to_history("assistant", reply)
            
            return reply
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            print(error_msg)
            return "Sorry, I encountered an error while generating a response."

# Note: We won't run this yet because it requires Ollama to be running
# ollama_handler = OllamaHandler()
# print(ollama_handler.generate("Hello, how are you today?"))

## 🔄 Step 6: Creating a Unified Handler

Now, let's combine everything into a unified handler that can work with both OpenAI and Ollama:

In [ ]:
class UnifiedLLMHandler:
    def __init__(self, provider="openai", model="gpt-4o-mini", name="PyPal", role="helpful AI assistant"):
        self.provider = provider.lower()
        self.model = model
        self.name = name
        self.role = role
        self.system_prompt = f"Your name is {name}, and you are a {role}. Always respond accordingly to this role."
        self.history = []
        
        # Check for API key if using OpenAI
        if self.provider == "openai":
            self.api_key = os.getenv("OPENAI_API_KEY")
            if not self.api_key:
                raise ValueError("OPENAI_API_KEY environment variable is required for OpenAI")
    
    def add_to_history(self, role, content):
        """Add a message to the conversation history."""
        self.history.append({"role": role, "content": content})
    
    def show_history(self):
        """Display the conversation history."""
        for entry in self.history:
            print(f"{entry['role'].capitalize()}: {entry['content']}")
    
    def generate(self, prompt):
        """Generate a response based on the configured provider."""
        if self.provider == "openai":
            return self._chat_with_openai(prompt)
        elif self.provider == "ollama":
            return self._chat_with_ollama(prompt)
        else:
            # Mock provider for testing
            mock_response = f"[MOCK {self.name}]: As a {self.role}, I respond to: {prompt}"
            self.add_to_history("user", prompt)
            self.add_to_history("assistant", mock_response)
            return mock_response
    
    def _chat_with_openai(self, prompt):
        """Generate a response using OpenAI's API."""
        url = "https://api.openai.com/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        # Get recent history (last 3 messages)
        recent = self.history[-3:] if len(self.history) >= 3 else self.history[:]
        
        # Prepare messages
        messages = [
            {"role": "system", "content": self.system_prompt}
        ]
        
        # Add conversation history
        messages.extend(recent)
        
        # Add current user message
        messages.append({"role": "user", "content": prompt})
        
        # Prepare the request data
        data = {
            "model": self.model,
            "messages": messages
        }
        
        try:
            # Make API request
            response = requests.post(url, headers=headers, json=data)
            response.raise_for_status()  # Raise exception for HTTP errors
            
            # Parse the response
            result = response.json()
            reply = result["choices"][0]["message"]["content"]
            
            # Add to history
            self.add_to_history("user", prompt)
            self.add_to_history("assistant", reply)
            
            return reply
        except Exception as e:
            error_msg = f"Error with OpenAI: {str(e)}"
            print(error_msg)
            return "Sorry, I encountered an error while generating a response."
    
    def _chat_with_ollama(self, prompt):
        """Generate a response using Ollama's API."""
        url = "http://localhost:11434/api/chat"
        
        # Get recent history (last 3 messages)
        recent = self.history[-3:] if len(self.history) >= 3 else self.history[:]
        
        # Prepare messages
        messages = [
            {"role": "system", "content": self.system_prompt}
        ]
        
        # Add conversation history
        messages.extend(recent)
        
        # Add current user message
        messages.append({"role": "user", "content": prompt})
        
        # Prepare the request data
        data = {
            "model": self.model,
            "messages": messages,
            "stream": False
        }
        
        try:
            # Make API request
            response = requests.post(url, json=data)
            response.raise_for_status()  # Raise exception for HTTP errors
            
            # Parse the response
            result = response.json()
            reply = result.get("message", {}).get("content", "No reply")
            
            # Add to history
            self.add_to_history("user", prompt)
            self.add_to_history("assistant", reply)
            
            return reply
        except Exception as e:
            error_msg = f"Error with Ollama: {str(e)}"
            print(error_msg)
            return "Sorry, I encountered an error while generating a response."

# Test with mock provider (no real API calls)
handler = UnifiedLLMHandler(provider="mock", name="Tester", role="debugging assistant")
print(handler.generate("Hello, can you help me debug my code?"))
print(handler.generate("I'm getting an IndexError."))
print("\nConversation History:")
handler.show_history()

## 🌟 Step 7: Creating a Simple Interface

Let's create a simple chat interface using our unified handler:

In [ ]:
def chat_with_llm(handler):
    print(f"Chatting with {handler.name} the {handler.role}")
    print("Type 'bye' to exit, 'history' to see conversation history")
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() in ["bye", "exit", "quit"]:
            print(f"{handler.name}: Goodbye! Have a great day!")
            break
        elif user_input.lower() == "history":
            print("\nConversation History:")
            handler.show_history()
            print()  # Empty line for better formatting
            continue
        
        # Generate response
        response = handler.generate(user_input)
        print(f"{handler.name}: {response}")

# We can run this with our mock handler
mock_handler = UnifiedLLMHandler(provider="mock", name="MockBot", role="coding assistant")
# Uncomment to test the chat interface:
# chat_with_llm(mock_handler)

## 🔍 Step 8: Refining Our Handler for Type Hints

Let's add type hints to our unified handler for better code quality:

In [ ]:
class SimpleLLMHandler:
    def __init__(
        self,
        provider: str = "openai",
        model: str = "gpt-4o-mini",
        name: str = "PyPal",
        role: str = "helpful AI assistant",
    ) -> None:
        self.provider = provider.lower()
        self.model = model
        self.name = name
        self.role = role
        self.system_prompt = f"Your name is {name}, and you are a {role}. Always respond accordingly to this role."
        self.history = []
        
        # Check for API key if using OpenAI
        if self.provider == "openai":
            self.api_key = os.getenv("OPENAI_API_KEY")
            if not self.api_key:
                raise ValueError("OPENAI_API_KEY environment variable is required for OpenAI")
    
    def add_to_history(self, role: str, content: str) -> None:
        """Add a message to the conversation history."""
        self.history.append({"role": role, "content": content})
    
    def show_history(self) -> None:
        """Display the conversation history."""
        for entry in self.history:
            print(f"{entry['role'].capitalize()}: {entry['content']}")
    
    def generate(self, prompt: str) -> str:
        """Generate a response based on the configured provider."""
        if self.provider == "openai":
            return self._chat_with_openai(prompt)
        elif self.provider == "ollama":
            return self._chat_with_ollama(prompt)
        else:
            # Mock provider for testing
            mock_response = f"[MOCK {self.name}]: As a {self.role}, I respond to: {prompt}"
            self.add_to_history("user", prompt)
            self.add_to_history("assistant", mock_response)
            return mock_response
    
    def _chat_with_openai(self, prompt: str) -> str:
        """Generate a response using OpenAI's API."""
        url = "https://api.openai.com/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        # Get recent history (last 3 messages)
        recent = self.history[-3:] if len(self.history) >= 3 else self.history[:]
        
        # Prepare messages
        messages = [
            {"role": "system", "content": self.system_prompt}
        ]
        
        # Add conversation history
        messages.extend(recent)
        
        # Add current user message
        messages.append({"role": "user", "content": prompt})
        
        # Prepare the request data
        data = {
            "model": self.model,
            "messages": messages
        }
        
        try:
            # Make API request
            response = requests.post(url, headers=headers, json=data)
            response.raise_for_status()  # Raise exception for HTTP errors
            
            # Parse the response
            result = response.json()
            reply = result["choices"][0]["message"]["content"]
            
            # Add to history
            self.add_to_history("user", prompt)
            self.add_to_history("assistant", reply)
            
            return reply
        except Exception as e:
            error_msg = f"Error with OpenAI: {str(e)}"
            print(error_msg)
            return "Sorry, I encountered an error while generating a response."
    
    def _chat_with_ollama(self, prompt: str) -> str:
        """Generate a response using Ollama's API."""
        url = "http://localhost:11434/api/chat"
        
        # Get recent history (last 3 messages)
        recent = self.history[-3:] if len(self.history) >= 3 else self.history[:]
        
        # Prepare messages
        messages = [
            {"role": "system", "content": self.system_prompt}
        ]
        
        # Add conversation history
        messages.extend(recent)
        
        # Add current user message
        messages.append({"role": "user", "content": prompt})
        
        # Prepare the request data
        data = {
            "model": self.model,
            "messages": messages,
            "stream": False
        }
        
        try:
            # Make API request
            response = requests.post(url, json=data)
            response.raise_for_status()  # Raise exception for HTTP errors
            
            # Parse the response
            result = response.json()
            reply = result.get("message", {}).get("content", "No reply")
            
            # Add to history
            self.add_to_history("user", prompt)
            self.add_to_history("assistant", reply)
            
            return reply
        except Exception as e:
            error_msg = f"Error with Ollama: {str(e)}"
            print(error_msg)
            return "Sorry, I encountered an error while generating a response."

# Test our final SimpleLLMHandler with the mock provider
final_handler = SimpleLLMHandler(provider="mock", name="FinalBot", role="Python tutor")
print(final_handler.generate("Can you explain what our LLM handler does?"))

## 📝 Exercise: Let's Practice!

Now that we've built our LLM handler from scratch, let's practice using it. Try the following exercises:

1. Create a `SimpleLLMHandler` with a custom name and role
2. Generate responses to a few different prompts
3. Check the conversation history
4. If you have an OpenAI API key or Ollama running, try connecting to a real LLM

In [ ]:
# Your exercise code here
# Example:
# my_handler = SimpleLLMHandler(provider="mock", name="MyBot", role="Data Science Expert")
# print(my_handler.generate("Can you explain what a neural network is?"))

## 🎉 Summary

In this notebook, we've built an LLM handler from scratch with the following features:

1. Support for multiple LLM providers (OpenAI and Ollama)
2. Conversation history tracking
3. System prompt customization
4. Error handling
5. Mock provider for testing

In the next notebook, we'll use this handler to connect our chatbot to real LLMs and see how it performs! 🚀